In [5]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
from io import BytesIO
import time

# 기본 URL 설정
base_url = "https://finance.naver.com/research/debenture_list.naver?searchType=keyword&keyword=%B1%DD%B8%AE&brokerCode=&writeFromDate=&writeToDate=&x=0&y=0"
total_pages = 10  # 총 페이지 수

# PDF 링크와 데이터를 저장할 딕셔너리
research_data = {}
pdf_links = []

# 1. 페이지 크롤링
for page in range(1, total_pages + 1):
    url = f"{base_url}&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 모든 tr 태그 가져오기
    rows = soup.find_all('tr')
    
    # td class="file" 안의 a 태그에서 PDF 링크 추출 및 정보 매핑
    file_tds = soup.find_all('td', class_='file')
    for i, td in enumerate(file_tds):
        # PDF 링크 추출
        pdf_link = td.find('a')['href']
        
        # 같은 행에서 제목과 날짜 추출 (i+1은 헤더 행을 건너뛰기 위함)
        if i + 1 < len(rows):
            row = rows[i + 1]
            
            # 제목 추출: <a href="debenture_read.naver?...">제목</a>
            title_elem = row.find('a')  # href가 debenture_read.naver로 시작하는 링크
            title = title_elem.text.strip() if title_elem else f"Unknown_{i}"
            
            # 날짜 추출: <td class="date">25.02.20</td>
            date_elem = row.find('td', class_='date')
            date = date_elem.text.strip() if date_elem else "Unknown"
        else:
            title = f"Unknown_{i}"
            date = "Unknown"
        
        pdf_links.append((pdf_link, title, date))
    
    print(f"페이지 {page} 크롤링 완료, 현재 {len(pdf_links)}개의 PDF 링크 수집")
    time.sleep(1)

# 2. PDF에서 텍스트 추출 및 딕셔너리에 저장
for idx, (pdf_url, title, date) in enumerate(pdf_links, 1):
    try:
        # PDF 스트리밍 다운로드
        pdf_response = requests.get(pdf_url, stream=True, timeout=10)
        pdf_file = BytesIO(pdf_response.content)
        
        # pdfplumber로 텍스트 추출
        with pdfplumber.open(pdf_file) as pdf:
            contents = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    contents += page_text
        
        # PDF 파일명에서 확장자 제거한 제목 추출
        pdf_title = pdf_url.split('/')[-1].replace('.pdf', '')
        
        # 딕셔너리에 저장
        research_data[title] = {
            'title': pdf_title,    # PDF 파일명에서 추출한 제목
            'contents': contents,  # PDF 내용
            'date': date          # 페이지에서 추출한 날짜
        }
        
        print(f"{idx}/{len(pdf_links)}: {title} 처리 완료")
    
    except Exception as e:
        print(f"{idx}/{len(pdf_links)}: {pdf_url} 처리 실패 - 오류: {e}")
    
    time.sleep(1)

print("모든 작업 완료!")
print(f"총 {len(research_data)}개의 리서치 데이터가 딕셔너리에 저장되었습니다.")

# 결과 예시 출력 (선택사항)
for title, data in list(research_data.items())[:3]:  # 처음 3개만 출력
    print(f"\nTitle (from page): {title}")
    print(f"PDF Title: {data['title']}")
    print(f"Date: {data['date']}")
    print(f"Contents preview: {data['contents'][:100]}...")

페이지 1 크롤링 완료, 현재 30개의 PDF 링크 수집
페이지 2 크롤링 완료, 현재 60개의 PDF 링크 수집
페이지 3 크롤링 완료, 현재 90개의 PDF 링크 수집
페이지 4 크롤링 완료, 현재 120개의 PDF 링크 수집
페이지 5 크롤링 완료, 현재 150개의 PDF 링크 수집
페이지 6 크롤링 완료, 현재 180개의 PDF 링크 수집
페이지 7 크롤링 완료, 현재 210개의 PDF 링크 수집
페이지 8 크롤링 완료, 현재 240개의 PDF 링크 수집
페이지 9 크롤링 완료, 현재 270개의 PDF 링크 수집
페이지 10 크롤링 완료, 현재 300개의 PDF 링크 수집
1/300: Unknown_0 처리 완료
2/300: 1분기 10년 하단 4.20% 처리 완료
3/300: 미국채 10년물 금리 하락을 견인한 텀프리미엄 처리 완료
4/300: [2월 금통위 Review] 확장재정이 Key 처리 완료
5/300: Daily Bond Morning Brief(2025.02.26) 처리 완료
6/300: Eugenes FICC Update 처리 완료
7/300: Unknown_6 처리 완료
8/300: Unknown_7 처리 완료
9/300: Unknown_8 처리 완료
10/300: 2월 금통위 리뷰: 통화정책과 재정정책 공조 필요 처리 완료
11/300: [IBKS Bond Inside] 2025년 2월 금통위 리뷰 처리 완료
12/300: 2월 금통위; 정책 공조의 시간 처리 완료
13/300: 2월 금통위 Review: 비둘기와 매를 모두, 최종금리 2... 처리 완료
14/300: [금통위 리뷰] 최소 1.5% 처리 완료
15/300: Unknown_14 처리 완료
16/300: Unknown_15 처리 완료
17/300: Unknown_16 처리 완료
18/300: 2월 금통위: 핵심 지표는 여전히 추가 금리인하를 지.. 처리 완료
19/300: 2월 금통위: 금리 인하 더 한다. 국고 금리 뉴레인지 .. 처리 완료
20/300: Daily B

In [2]:
pip install pdfplumber


  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.5-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl (3.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


페이지 1 크롤링 완료, 현재 30개의 데이터 수집
1/30: Unknown_0 처리 완료
2/30: 1분기 10년 하단 4.20% 처리 완료
3/30: 미국채 10년물 금리 하락을 견인한 텀프리미엄 처리 완료
4/30: [2월 금통위 Review] 확장재정이 Key 처리 완료
5/30: Daily Bond Morning Brief(2025.02.26) 처리 완료
6/30: Eugenes FICC Update 처리 완료
7/30: Unknown_6 처리 완료
8/30: Unknown_7 처리 완료
9/30: Unknown_8 처리 완료
10/30: 2월 금통위 리뷰: 통화정책과 재정정책 공조 필요 처리 완료
11/30: [IBKS Bond Inside] 2025년 2월 금통위 리뷰 처리 완료
12/30: 2월 금통위; 정책 공조의 시간 처리 완료
13/30: 2월 금통위 Review: 비둘기와 매를 모두, 최종금리 2... 처리 완료
14/30: [금통위 리뷰] 최소 1.5% 처리 완료
15/30: Unknown_14 처리 완료
16/30: Unknown_15 처리 완료
17/30: Unknown_16 처리 완료
18/30: 2월 금통위: 핵심 지표는 여전히 추가 금리인하를 지.. 처리 완료
19/30: 2월 금통위: 금리 인하 더 한다. 국고 금리 뉴레인지 .. 처리 완료
20/30: Daily Bond Morning Brief(2025.02.25) 처리 완료
21/30: Eugenes FICC Update 처리 완료
22/30: 브라질, 연초 효과 지속 여부, 재정에 달렸다 처리 완료
23/30: Unknown_22 처리 완료
24/30: Unknown_23 처리 완료
25/30: Unknown_24 처리 완료
26/30: 채권 Daily (02.24) 처리 완료
27/30: [FI Weekly] 금통위 Preview: 만장일치 인하 처리 완료
28/30: Daily Bond Morning Brief(2025.02.24) 처리 완료
29/30: Eugene

In [8]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
from io import BytesIO
import time
import csv

# 기본 URL 설정
base_url = "https://finance.naver.com/research/debenture_list.naver?searchType=keyword&keyword=%B1%DD%B8%AE&brokerCode=&writeFromDate=&writeToDate=&x=0&y=0"
total_pages = 1  # 총 페이지 수 (테스트용으로 1로 바꿀 수 있음)

# 데이터를 저장할 리스트
research_data = []

# 날짜 형식을 YYYY-MM-DD로 변환하는 함수
def format_date(date_str):
    try:
        year, month, day = date_str.split('.')
        full_year = f"20{year}"
        return f"{full_year}-{month.zfill(2)}-{day.zfill(2)}"
    except:
        return "Unknown"

# 1. 페이지 크롤링
for page in range(1, total_pages + 1):
    url = f"{base_url}&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    rows = soup.find_all('tr')
    file_tds = soup.find_all('td', class_='file')
    for i, td in enumerate(file_tds):
        pdf_link = td.find('a')['href']
        
        if i + 1 < len(rows):
            row = rows[i + 1]
            title_elem = row.find('a')
            title = title_elem.text.strip() if title_elem else f"Unknown_{i}"
            date_elem = row.find('td', class_='date')
            raw_date = date_elem.text.strip() if date_elem else "Unknown"
            date = format_date(raw_date)
        else:
            title = f"Unknown_{i}"
            date = "Unknown"
        
        research_data.append({
            'title': title,
            'date': date,
            'pdf_url': pdf_link,  # 임시로 저장
            'contents': ''
        })
    
    print(f"페이지 {page} 크롤링 완료, 현재 {len(research_data)}개의 데이터 수집")
    time.sleep(1)

# 2. PDF에서 텍스트 추출 (텍스트가 없으면 스킵)
final_data = []  # 텍스트가 있는 데이터만 저장할 최종 리스트
for idx, item in enumerate(research_data, 1):
    pdf_url = item['pdf_url']
    try:
        pdf_response = requests.get(pdf_url, stream=True, timeout=10)
        pdf_file = BytesIO(pdf_response.content)
        
        with pdfplumber.open(pdf_file) as pdf:
            contents = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    contents += page_text.replace('\n', ' ')  # 한 줄로 변환
        
        # 텍스트가 있는 경우에만 추가
        if contents.strip():  # 공백만 있는 경우도 제외
            item['contents'] = contents
            del item['pdf_url']  # url 제거
            final_data.append(item)
            print(f"{idx}/{len(research_data)}: {item['title']} 처리 완료")
        else:
            print(f"{idx}/{len(research_data)}: {item['title']} 스킵 - 텍스트 없음")
    
    except Exception as e:
        print(f"{idx}/{len(research_data)}: {pdf_url} 처리 실패 - 오류: {e}")
    
    time.sleep(1)

# 3. CSV 파일로 저장 (텍스트가 있는 데이터만)
csv_file = "research_data.csv"
with open(csv_file, 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['title', 'contents', 'date'])
    writer.writeheader()
    writer.writerows(final_data)

print("모든 작업 완료!")
print(f"총 {len(final_data)}개의 유효 데이터가 {csv_file}에 저장되었습니다.")

페이지 1 크롤링 완료, 현재 30개의 데이터 수집
1/30: Unknown_0 처리 완료
2/30: 1분기 10년 하단 4.20% 처리 완료
3/30: 미국채 10년물 금리 하락을 견인한 텀프리미엄 처리 완료
4/30: [2월 금통위 Review] 확장재정이 Key 처리 완료
5/30: Daily Bond Morning Brief(2025.02.26) 처리 완료
6/30: Eugenes FICC Update 처리 완료
7/30: Unknown_6 처리 완료
8/30: Unknown_7 처리 완료
9/30: Unknown_8 처리 완료
10/30: 2월 금통위 리뷰: 통화정책과 재정정책 공조 필요 처리 완료
11/30: [IBKS Bond Inside] 2025년 2월 금통위 리뷰 처리 완료
12/30: 2월 금통위; 정책 공조의 시간 처리 완료
13/30: 2월 금통위 Review: 비둘기와 매를 모두, 최종금리 2... 처리 완료
14/30: [금통위 리뷰] 최소 1.5% 처리 완료
15/30: Unknown_14 처리 완료
16/30: Unknown_15 처리 완료
17/30: Unknown_16 처리 완료
18/30: 2월 금통위: 핵심 지표는 여전히 추가 금리인하를 지.. 처리 완료
19/30: 2월 금통위: 금리 인하 더 한다. 국고 금리 뉴레인지 .. 처리 완료
20/30: Daily Bond Morning Brief(2025.02.25) 처리 완료
21/30: Eugenes FICC Update 처리 완료
22/30: 브라질, 연초 효과 지속 여부, 재정에 달렸다 처리 완료
23/30: Unknown_22 처리 완료
24/30: Unknown_23 처리 완료
25/30: Unknown_24 처리 완료
26/30: 채권 Daily (02.24) 처리 완료
27/30: [FI Weekly] 금통위 Preview: 만장일치 인하 처리 완료
28/30: Daily Bond Morning Brief(2025.02.24) 처리 완료
29/30: Eugene